# XX公司贷款审批预测
## 项目背景
XX是一家贷款公司，承接个人贷款业务。然而，贷款人员鱼龙混杂，能力不一。贷款人可能会因为各种原因，导致逾期甚至无力偿还贷款。XX公司也曾因为贷款人的各种原因，造成公司一定程度的损失。  
随着XX公司的发展壮大，每年贷款的人数也越来越多，由于贷款对审核人员的要求较高，既需要很强的行业背景知识，也需要一定的经验辅助。因此，大量的贷款审批要求XX公司雇佣大量审批员工。这给XX公司带来了一定的困扰：
* 廉价的审批人员专业能力不强，可能不能正确的发现无力偿还贷款的人员。
* 有经验的员工，成本又非常高。

## 数据集描述
* id 贷款编号。
* member_id 会员编号。
* loan_amnt 借款人申请的贷款金额。
* funded_amnt 承诺给该贷款的总金额。
* funded_amnt_inv 投资者为该贷款承诺的总金额。
* term 贷款的偿还时间。
* int_rate 贷款的利率。
* installment 分期付款，每期还款的额度。
* grade 贷款等级。贷款利率越高，则等级越高。
* sub_grade 贷款子等级。
* emp_title 工作名称。
* emp_length 工作时间。
* home_ownership 房屋所有权状态。取值为：
    + RENT 出租
    + OWN 自由
    + MORTGAGE 按揭
    + OTHER 其他
* annual_inc 贷款人自报的年收入。
* verification_status 贷款人收入是否核实。
* issue_d 贷款月份。
* loan_status 贷款的当前状态。
* pymnt_plan 是否已经为贷款实施还款计划。
* url 贷款的url地址。
* desc 贷款人的贷款描述。
* purpose 贷款人贷款的用途。
* title 贷款人提供的标题。
* zip_code 邮政编码
* addr_state 贷款人所在的国家。
* dti 贷款人的总债务偿还总额与贷款人的月收入比值。
* delinq_2yrs 过去两年借款人信用档案中逾期30天以上的拖欠事件。
* earliest_cr_line 借款人最早报告的信贷额度开始的月份。
* inq_last_6mths 过去六个月的查询数目。
* open_acc 借款人信用档案中的未结信用额度。
* pub_rec 贬损公共记录的数量。
* revol_bal 总信贷周转余额。

## 加载数据
通过pandas读取csv数据集文件。

In [2]:
import pandas as pd
import numpy as np

# 数据的第一行是描述信息，因此使用skiprows跳过。第二行才是标题数据。
# 数据类型不同时，显式执行low_memory=False。会更好些。

# 参数：skiprows设置要跳过的行。参数值可以是int或list类型。
# 当为int类型时，表示要跳过前n行。
# 当为list类型时，表示要跳过行的索引。（索引从0开始）

# low_memory 参数默认值为True，在读取数据的时候，默认使用数据块中进行处理。这样可以降低
# 内容使用量。此种方式仅适合列数较少，且类型单一的情况。
data = pd.read_csv("LoanStats3a.csv", low_memory=False, skiprows=1)
# data = pd.read_csv("LoanStats3a.csv", low_memory=False, skiprows=[5, 10])

## 简要浏览数据
对数据进行查看。可以使用head，tail，sample等方法。

In [3]:
data.sample(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
29990,516506,667562.0,1500.0,1500.0,1500.0,36 months,13.85%,51.16,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
16535,731445,927513.0,15000.0,15000.0,15000.0,60 months,17.14%,373.92,E,E3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
23402,615402,789157.0,5600.0,5600.0,5500.0,36 months,14.46%,192.65,D,D2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
13118,787453,990994.0,4000.0,4000.0,3750.0,36 months,7.49%,124.41,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
12699,782440,985428.0,30000.0,30000.0,29975.0,36 months,15.62%,1049.10,D,D1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [6]:
# 默认情况下，只显示20列数据。
print(pd.get_option("max_columns"))
# 查看数据集的列数。
print(data.shape)
pd.set_option("max_columns", 120)
data.head(3)

20
(42538, 111)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.0,0.00,Jan-2015,171.62,NaN,Nov-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.0,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.0,0.0,0.00,Jun-2014,649.91,NaN,Nov-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


## 数据预处理
### 直观删除特征
通过观看数据集，可以直观上删除一些特征：
* 与贷款没有关联的特征，如id，memeber_id。
* 特征之间相关度极高，可以只保留一个即可。例如grade, sub_grade与int_rate，只保留int_rate即可。
* 除了缺失值外，所有的值都相同，则该特征可以删除。

In [3]:
# 删除无关特征。;';'
irrelevant_coumns = ["id", "member_id", "funded_amnt", "funded_amnt_inv", "emp_title", "issue_d",
        "url", "desc", "zip_code", "addr_state", "last_credit_pull_d", "earliest_cr_line", "addr_state",
        "title", "last_pymnt_d"]
# axis 0（删除行） 1（删除列），默认为0。
data.drop(irrelevant_coumns, axis=1, inplace=True)
# 删除相关性高的特征。
high_relevant = ["grade", "sub_grade"]
data.drop(high_relevant, axis=1, inplace=True)
# 创建一个列表，列表中存储去空值之后，仅有一个值的列。然后
# 统一删除这些列。
only_one = []
# 删除除空值外只有一个值的特征。
for c in data.columns:
    # 某列（Series）在去除空值之后，仅剩下一个值。
    if len(data[c].dropna().unique()) == 1:
        only_one.append(c)
data.drop(only_one, axis=1, inplace=True)
# 查看经过删除特征之后，还剩多少。
print(data.shape[1])

90


### 数据清洗
数据清洗通常包含：
* 缺失值处理
* 异常值处理
* 重复值处理

### 缺失值
* 可以通过info方法查看缺失值。
* 可以借助于isnull，notnull与sum等函数查看缺失值。

In [1]:
# 当数据量大时，一些信息将不再显示。这里设置相关参数为True。
# 目的是为了显示信息。
data.info(null_counts=True, verbose=True)

NameError: name 'data' is not defined

In [4]:
# 默认情况下，只显示60行数据。
# print(pd.get_option("max_rows"))
# pd.set_option("max_rows", 200)
data.isnull().sum(axis=0)

loan_amnt                             3
term                                  3
int_rate                              3
installment                           3
emp_length                         1115
home_ownership                        3
annual_inc                            7
verification_status                   3
loan_status                           3
pymnt_plan                            3
purpose                               3
dti                                   3
delinq_2yrs                          32
inq_last_6mths                       32
mths_since_last_delinq            26929
mths_since_last_record            38887
open_acc                             32
pub_rec                              32
revol_bal                             3
revol_util                           93
total_acc                            32
out_prncp                             3
out_prncp_inv                         3
total_pymnt                           3
total_pymnt_inv                       3


通过运行结果发现，在数据集中，部分特征缺失比例过多，直接删除该特征即可。我们这里将非空的阈值设置为2 / 3。

In [5]:
# 计算非空值的阈值。
thresh = int(len(data) * 2 / 3)
# 删除非空值低于阈值的列。
data.dropna(axis=1, thresh=thresh, inplace=True)
print(data.shape[1])

33


删除缺失值较多的特征之后，再来查看缺失值情况。

In [6]:
# 查看缺失值的比例。
print(data.isnull().sum() / len(data))
# data.isnull().sum()

loan_amnt                  0.000071
term                       0.000071
int_rate                   0.000071
installment                0.000071
emp_length                 0.026212
home_ownership             0.000071
annual_inc                 0.000165
verification_status        0.000071
loan_status                0.000071
pymnt_plan                 0.000071
purpose                    0.000071
dti                        0.000071
delinq_2yrs                0.000752
inq_last_6mths             0.000752
open_acc                   0.000752
pub_rec                    0.000752
revol_bal                  0.000071
revol_util                 0.002186
total_acc                  0.000752
out_prncp                  0.000071
out_prncp_inv              0.000071
total_pymnt                0.000071
total_pymnt_inv            0.000071
total_rec_prncp            0.000071
total_rec_int              0.000071
total_rec_late_fee         0.000071
recoveries                 0.000071
collection_recovery_fee    0

这时，缺失值占数据集总数的比例较少，直接删除缺失值。

In [7]:
data.dropna(axis=0, how="any", inplace=True)
# data.isnull().sum()
display(data.shape)

(40003, 33)

### 异常值
可以通过descirbe方法查看数据统计信息。

In [20]:
data.describe()

,loan_amnt,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens
count,40003.000000,40003.000000,4.000300e+04,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,4.000300e+04,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.000000,40003.0,40003.0,40003.000000,40003.0
mean,11232.427568,325.718770,6.992762e+04,13.440492,0.149639,1.039547,9.376847,0.056871,1.432637e+04,22.285854,2.893927,2.886769,12216.118148,11691.640602,9827.870118,2286.713826,1.447278,100.087019,13.475782,2682.540521,0.0,0.0,0.043597,0.0
std,7443.612602,209.201013,6.398381e+04,6.678122,0.501641,1.385258,4.436273,0.241739,2.158238e+04,11.514349,47.632374,47.510211,9153.003702,9039.345347,7142.715510,2622.720593,7.594400,709.383027,155.470054,4442.427374,0.0,0.0,0.205176,0.0
min,500.000000,15.670000,3.300000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
25%,5500.000000,167.770000,4.162800e+04,8.320000,0.000000,0.000000,6.000000,0.000000,3.761000e+03,14.000000,0.000000,0.000000,5568.717914,5197.685000,4500.000000,672.875000,0.000000,0.000000,0.000000,218.655000,0.0,0.0,0.000000,0.0
50%,10000.000000,280.830000,6.000000e+04,13.530000,0.000000,1.000000,9.000000,0.000000,8.990000e+03,21.000000,0.000000,0.000000,9899.493931,9384.120000,8000.000000,1368.780000,0.000000,0.000000,0.000000,552.770000,0.0,0.0,0.000000,0.0
75%,15000.000000,432.260000,8.400000e+04,18.710000,0.000000,2.000000,12.000000,0.000000,1.740200e+04,29.000000,0.000000,0.000000,16637.410583,15981.240000,14000.000000,2867.880000,0.000000,0.000000,0.000000,3308.090000,0.0,0.0,0.000000,0.0
max,35000.000000,1305.190000,6.000000e+06,29.990000,11.000000,17.000000,46.000000,4.000000,1.207359e+06,90.000000,3126.610000,3123.440000,58872.160000,58563.680000,35000.020000,23878.150000,209.000000,29623.350000,7002.190000,36115.200000,0.0,0.0,2.000000,0.0


### 重复值
可通过duplicated方法查看重复值数量。可以通过布尔数组提取元素的方式，查看重复值的具体信息。  
可通过drop_duplicates方法删除重复值。

In [8]:
data.duplicated().sum()
# data.drop_duplicates(inplace=True)

0

## 数据转换
机器学习模型接收数值类型的数据，因此，为了能够将数据输入模型中进行训练，需要将非数值类型转换为数值类型。

### 查看所有object类型的列
在本例中，int与float类型的数据无需处理，而object类型的数据需要转换为数值类型。  
首先查看下，哪些列为object类型。

In [9]:
# 返回DataFrame的一个子集。仅包含复合条件的列类型。
# 仅选择类型为object类型的列。
# 返回的依然还是一个DataFrame类型。
t = data.select_dtypes(include=["object"])
print(t.columns)
type(t)
# t

Index(['term', 'int_rate', 'emp_length', 'home_ownership',
       'verification_status', 'loan_status', 'pymnt_plan', 'purpose',
       'revol_util'],
      dtype='object')


pandas.core.frame.DataFrame

### 按需求对列进行转换
可调用每个列的value_counts方法，统计每一列可能的取值与数量。  
`data[column_name].value_counts()`  

### pymnt_plan
pymnt_plan列值为y的记录仅有1条，该列可以删除。

In [27]:
# data["pymnt_plan"].value_counts()
data.drop("pymnt_plan", axis=1, inplace=True)

### int_rate与revol_util
int_rate与revol_util含有数值，但是具有%，可以去掉%值。

In [30]:
# 也可以使用map或者apply方法来实现相同的功能。
data["int_rate"] = data["int_rate"].str.replace("%", "").astype(np.float)
data["revol_util"] = data["revol_util"].str.replace("%", "").astype(np.float)

### emp_length
该列含有数值，这里需要转换为对应的数值类型。

In [33]:
data["emp_length"].value_counts()

10+ years    9169
< 1 year     4701
2 years      4576
3 years      4242
4 years      3550
5 years      3389
1 year       3373
6 years      2310
7 years      1830
8 years      1553
9 years      1310
Name: emp_length, dtype: int64

In [34]:
map_dict = {
    "10+ years": 10, "9 years": 9, "8 years": 8,
    "7 years": 7, "6 years": 6, "5 years": 5,
    "4 years": 4, "3 years": 3, "2 years": 2,
    "1 year": 1, "< 1 year": 0
}
data["emp_length"] = data["emp_length"].map(map_dict)

### loan_status
loan_status列为标签列，该列含有若干个取值，我们仅取其中的两个：
* Fully Paid 符合条件，已经放款
* Charged Off 不符合条件，拒绝。

**说明：由于缺少真实的数据，这里我们假设所有拒绝放款的记录，就是不能正常偿还贷款的用户。**

In [38]:
# data["loan_status"].value_counts()
# 仅选择值为Fully Paid与loan_status的记录。
data = data[(data["loan_status"] == "Fully Paid") | (data["loan_status"] == "Charged Off")]
data["loan_status"] = data["loan_status"].map({"Fully Paid": 1, "Charged Off": 0})
data["loan_status"].value_counts()

d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


1    32448
0     5295
Name: loan_status, dtype: int64

### 其余列
其余的几列就是单纯的离散值，没有顺序大小之分，这里我们使用one-hot来对其进行处理。

In [39]:
data["home_ownership"].value_counts()

RENT        17996
MORTGAGE    16883
OWN          2768
OTHER          96
Name: home_ownership, dtype: int64

In [44]:
columns = ["home_ownership", "verification_status", "purpose", "term"]
# 将data[columns]中每一列转换成独热编码（one-hot）编码的形式。
dummy = pd.get_dummies(data[columns])
# 将独热编码与原data数据进行组合在一起。
data = pd.concat([data, dummy], axis=1)
# 删除转换为独热编码的列。
data = data.drop(columns, axis=1)

In [42]:
# data[["home_ownership"]]
# pd.get_dummies(data[["home_ownership"]])

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37743 entries, 0 to 39749
Data columns (total 51 columns):
loan_amnt                              37743 non-null float64
int_rate                               37743 non-null float64
installment                            37743 non-null float64
emp_length                             37743 non-null int64
annual_inc                             37743 non-null float64
loan_status                            37743 non-null int64
dti                                    37743 non-null float64
delinq_2yrs                            37743 non-null float64
inq_last_6mths                         37743 non-null float64
open_acc                               37743 non-null float64
pub_rec                                37743 non-null float64
revol_bal                              37743 non-null float64
revol_util                             37743 non-null float64
total_acc                              37743 non-null float64
out_prncp                  

## 使用逻辑回归进行二分类

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, classification_report

lr = LogisticRegression()
y = data["loan_status"]
# 数据集中，删除便签y列，剩下的特征就是X。
X = data.drop("loan_status", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
print(classification_report(y_test, y_hat))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99      1341
          1       1.00      1.00      1.00      8095

avg / total       1.00      1.00      1.00      9436



## 样本不均衡

In [ ]:
y.value_counts()

# 项目经验
项目名称 为了避免每个人的名称相同，可以考虑  地点+公司（机构）+ 项目用途

写项目必须首先介绍项目背景——为什么要启动该项目（项目的需求），该项目能够为人们带来怎样的价值（有什么用）。  
项目时间： 可以不写在简历中，但是心中要有一个具体的时间。  
项目实现： 项目开发环境 可以考虑加上开发的版本  例如： numpy 1.14.0。 注意：版本号最好不要使用最新的。  
技术实现  要写一些有用的内容。不要写一些大众化的流程。例如：pd.read_csv读取数据。类似这样的内容，一笔带过即可。  


示例：  
* 加载并且读取数据集，并进行数据清洗（缺失值，异常值，重复值）。  
* 进行特征删除的工作：  
    1. 删除直观上与预测结果无关的特征。
    2. 删除特征相关性较高的特征。（只保留一个）
    3. 删除除缺失值外，仅有一个值的特征。 
* 进行数据转换与特征工程。将非数值特征转换为数值类型。包括apply,map与one-hot编码。
* 使用逻辑回归LogisticRegresion进行数据建模。【建议使用多个模型进行尝试，然后选择一个效果较好的模型。】

项目结果（项目的评估指标）：
项目要给出运行结果，对于回归，应该包含$R^2$值，或MSE，MAE等。对于分类，应该体现出精准率与召回率，F1调和平均值。  
对于二分类任务，可以画出ROC曲线，求出AUC面积值。

项目价值：
项目结果为客户（本公司）带来了怎样的价值。  
示例：  
通过此项目，成功预测出绝大部分不能偿还贷款的人员，从而及时阻止为没有偿还能力的贷款人放款，进而减少公司的损失。

描述项目价值时，我们可以采用前后对比的方式来进行描述。包括：
* 在使用模型之前，是什么样的状态。
* 在使用模型之后，是什么样的状态。

示例：  
在使用模型之前，使用人工的方式进行贷款偿还预测，准确率较低，及时性较差。  
* 及时性：平均贷款人要等待3-5天，才能收到放款。使用模型之后，平均1-2天就能收到放款。
* 准确性：以前人工预测，每放款1000人，有8-10人不能如期偿还贷款，使用模型预测后，每放款1000人，有3-5人不能如期偿还贷款。


http://playground.tensorflow.org